| Stage 2: Supervised Fine-tuning | 有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图   | [scripts/supervised_finetuning.py](https://github.com/shibing624/MedicalGPT/blob/main/scripts/supervised_finetuning.py) | [scripts/run_sft.sh](https://github.com/shibing624/MedicalGPT/blob/main/scripts/run_sft.sh)  | [notebook/run_supervised_finetuning.ipynb](https://github.com/shibing624/MedicalGPT/blob/main/notebook/run_supervised_finetuning.ipynb)     | [Open In Colab](https://colab.research.google.com/github/shibing624/MedicalGPT/blob/main/notebook/run_supervised_finetuning.ipynb) |

# Stage 2: Supervised FineTuning

第二阶段：SFT(Supervised Fine-tuning)有监督微调，构造指令微调数据集，在预训练模型基础上做指令精调，以对齐指令意图


#### 说明：
以下 notebook/colab 代码为了快速验证训练代码可用，我们使用了小size的生成模型和小样本数据集，实际使用时，需要使用更大的模型和数据集，以获得更好的效果。

1. 生成模型：使用的是Bloom的`bigscience/bloomz-560m`
2. 数据集：SFT阶段使用的是使用的是Belle的1千条抽样数据，位于`data/finetune`文件夹

## 配置运行环境

In [1]:
# 本地执行可注释以下配置环境的命令，colab执行要打开注释，用于配置环境

# !git clone --depth 1 https://github.com/shibing624/MedicalGPT.git
# %cd MedicalGPT
# %ls

安装库和依赖包：

```
loguru
transformers>=4.28.1
datasets
tensorboard
tqdm>=4.47.0
peft>=0.3.0
trl
```

In [2]:
# !pip install -r requirements.txt

In [3]:
# %cd notebook

## 咱们开始吧

环境配置完成，开始导入包

In [4]:
# Copyright 2023 XuMing(xuming624@qq.com) and The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for causal language modeling (GPT, GPT-2, CTRL, ...) on a text file or a dataset.

part of this code is adapted from https://github.com/shibing624/textgen/blob/main/textgen/llama/llama_model.py
"""
import math
import os
from dataclasses import dataclass, field
from glob import glob
from typing import Optional

import torch
from datasets import load_dataset
from loguru import logger
from peft import LoraConfig, TaskType, get_peft_model, PeftModel, prepare_model_for_int8_training
from transformers import (
    BloomForCausalLM,
    AutoModel,
    LlamaTokenizer,
    LlamaForCausalLM,
    BloomTokenizerFast,
    AutoTokenizer,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
    DataCollatorForSeq2Seq,
)
from transformers.trainer import TRAINING_ARGS_NAME
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import send_example_telemetry

2023-06-07 17:42:46.393028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 17:42:46.567745: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 17:42:48.291399: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-07 17:42:48.291621: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/flemingxu/disk/py38/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/flemingxu/disk/py38/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('disk/nlp/MedicalGPT/notebook/run_supervised_finetuning.ipynb')}
  warn(msg)
/home/flemingxu/disk/py38/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)
/home/flemingxu/disk/py38/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/flemingxu/disk/py38/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: No libcudart.so found! Install CUDA or the cudatoolkit package (anaconda)!
  warn(msg)
/home/flem

In [5]:
MODEL_CLASSES = {
    "bloom": (BloomForCausalLM, BloomTokenizerFast),
    "chatglm": (AutoModel, AutoTokenizer),
    "llama": (LlamaForCausalLM, LlamaTokenizer),
}

IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
PROMPT_TEMPLATE = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n\n"
    "### Instruction:\n{instruction}\n\n### Response: "
)


## 指定参数

In [6]:
%ls ../data/finetune

Belle_open_source_1k.json


In [7]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_type: str = field(
        default='bloom',
        metadata={"help": "Model type selected in the list: " + ", ".join(MODEL_CLASSES.keys())}
    )
    model_name_or_path: Optional[str] = field(
        default="bigscience/bloomz-560m",
        metadata={
            "help": (
                "The model checkpoint for weights initialization.Don't set if you want to train a model from scratch."
            )
        },
    )
    tokenizer_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The tokenizer for weights initialization.Don't set if you want to train a model from scratch."
            )
        },
    )
    load_in_8bit: bool = field(default=False, metadata={"help": "Whether to load the model in 8bit mode or not."})
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=False,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    torch_dtype: Optional[str] = field(
        default="float16",
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype. If `auto` is passed, the "
                "dtype will be automatically derived from the model's weights."
            ),
            "choices": ["auto", "bfloat16", "float16", "float32"],
        },
    )
    device_map: Optional[str] = field(
        default="auto",
        metadata={"help": "Device to map model to. If `auto` is passed, the device will be selected automatically. "},
    )
    trust_remote_code: bool = field(
        default=True,
        metadata={"help": "Whether to trust remote code when loading a model from a remote checkpoint."},
    )

    def __post_init__(self):
        if self.model_type is None:
            raise ValueError(
                "You must specify a valid model_type to run training. Available model types are " + ", ".join(
                    MODEL_CLASSES.keys()))
        if self.model_name_or_path is None:
            raise ValueError("You must specify a valid model_name_or_path to run training.")


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file_dir: Optional[str] = field(default="../data/finetune", metadata={"help": "The train jsonl data file folder."})
    validation_file_dir: Optional[str] = field(default="../data/finetune", metadata={"help": "The evaluation jsonl file folder."})
    max_source_length: Optional[int] = field(default=256, metadata={"help": "Max length of prompt input text"})
    max_target_length: Optional[int] = field(default=256, metadata={"help": "Max length of output text"})
    max_train_samples: Optional[int] = field(
        default=1000,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of training examples to this "
                "value if set."
            )
        },
    )
    max_eval_samples: Optional[int] = field(
        default=10,
        metadata={
            "help": (
                "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
                "value if set."
            )
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[float] = field(
        default=0.05,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )


@dataclass
class PeftArguments(TrainingArguments):
    use_peft: bool = field(default=True, metadata={"help": "Whether to use peft"})
    target_modules: Optional[str] = field(default="all")
    lora_rank: Optional[int] = field(default=8)
    lora_dropout: Optional[float] = field(default=0.05)
    lora_alpha: Optional[float] = field(default=32.0)
    modules_to_save: Optional[str] = field(default=None)
    peft_path: Optional[str] = field(default=None)
    
    # notebook 直接写进参数
    output_dir: str = field(default='outputs-sft')
    do_train: bool = field(default=True)
    do_eval: bool = field(default=True)
    fp16: bool = field(default=True)
    num_train_epochs: float = field(default=0.5)
    learning_rate: float = field(default=2e-5)
    warmup_steps: int = field(default=10)
    weight_decay: float = field(default=0.01)
    logging_strategy: str = field(default='steps')
    logging_steps: int = field(default=10)
    evaluation_strategy: str = field(default='steps')
    eval_steps: int = field(default=10)
    save_strategy: str = field(default='steps')
    save_steps: int = field(default=500)
    save_total_limit: int = field(default=3)
    seed: int = field(default=42)
    per_device_train_batch_size: int = field(default=4)
    per_device_eval_batch_size: int = field(default=4)
    overwrite_output_dir: bool = field(default=True)
    logging_first_step: bool = field(default=True)
    report_to: str = field(default="tensorboard")
    gradient_checkpointing: bool = field(default=True)

In [8]:
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = PeftArguments()

training_args

PeftArguments(output_dir='outputs-sft', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=0.5, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.0, warmup_steps=10, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='outputs-sft/runs/Jun07_17-43-30_ts-be7e7b4bf1f24d21bb9c3d60ec0c1ae2-launcher', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=True, logging_steps=10, logging_nan_inf_filter=True, save_strategy=<IntervalStrategy.STEPS: 'steps'>, save_steps=500, save_total_limit

In [9]:
send_example_telemetry("run_sft", model_args, data_args)

logger.warning(f"Model args: {model_args}")
logger.warning(f"Data args: {data_args}")
logger.warning(f"Training args: {training_args}")
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)

2023-06-07 17:43:30.762 | WARNING  | __main__:<cell line: 3>:3 - Model args: ModelArguments(model_type='bloom', model_name_or_path='../../../models/bigscience/bloomz-560m', tokenizer_name_or_path=None, load_in_8bit=False, cache_dir=None, use_fast_tokenizer=False, torch_dtype='float16', device_map='auto', trust_remote_code=True)
2023-06-07 17:43:30.763 | WARNING  | __main__:<cell line: 4>:4 - Data args: DataTrainingArguments(dataset_name=None, dataset_config_name=None, train_file_dir='../data/finetune', validation_file_dir='../data/finetune', max_source_length=256, max_target_length=256, max_train_samples=1000, max_eval_samples=10, overwrite_cache=False, validation_split_percentage=0.05, preprocessing_num_workers=None)
2023-06-07 17:43:30.765 | WARNING  | __main__:<cell line: 5>:5 - Training args: PeftArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_la

## 定义各函数

In [10]:
class CastOutputToFloat(torch.nn.Sequential):
    """Cast the output of the model to float"""

    def forward(self, x):
        return super().forward(x).to(torch.float32)


class SavePeftModelTrainer(Trainer):
    """
    Trainer for lora models
    """

    def save_model(self, output_dir=None, _internal_call=False):
        """Save the LoRA model."""
        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        self.model.save_pretrained(output_dir)


def save_model(output_dir, model, tokenizer, args):
    """Save the model and the tokenizer."""
    os.makedirs(output_dir, exist_ok=True)

    # Take care of distributed/parallel training
    model_to_save = model.module if hasattr(model, "module") else model
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    torch.save(args, os.path.join(output_dir, TRAINING_ARGS_NAME))


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


def find_all_linear_names(peft_model, int4=False, int8=False):
    """Find all linear layer names in the model. reference from qlora paper."""
    cls = torch.nn.Linear
    if int4 or int8:
        import bitsandbytes as bnb
        if int4:
            cls = bnb.nn.Linear4bit
        elif int8:
            cls = bnb.nn.Linear8bitLt
    lora_module_names = set()
    for name, module in peft_model.named_modules():
        if isinstance(module, cls):
            # last layer is not add to lora_module_names
            if 'lm_head' in name:
                continue
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    return sorted(lora_module_names)

In [11]:
set_seed(training_args.seed)

## 加载模型和tokenizer

In [12]:
# Load model
if not model_args.model_type:
    raise ValueError("Please specify a model_type, e.g. llama, chatglm, bloom, etc.")
model_class, tokenizer_class = MODEL_CLASSES[model_args.model_type]
if model_args.model_name_or_path:
    torch_dtype = (
        model_args.torch_dtype
        if model_args.torch_dtype in ["auto", None]
        else getattr(torch, model_args.torch_dtype)
    )
    model = model_class.from_pretrained(
        model_args.model_name_or_path,
        load_in_8bit=model_args.load_in_8bit,
        cache_dir=model_args.cache_dir,
        torch_dtype=torch_dtype,
        device_map=model_args.device_map,
        trust_remote_code=model_args.trust_remote_code,
    )
    model.lm_head = CastOutputToFloat(model.lm_head)
else:
    raise ValueError(f"Error, model_name_or_path is None, SFT must be loaded from a pre-trained model")

model

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (

In [13]:
# Load tokenizer
tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "trust_remote_code": model_args.trust_remote_code,
}
tokenizer_name_or_path = model_args.tokenizer_name_or_path
if not tokenizer_name_or_path:
    tokenizer_name_or_path = model_args.model_name_or_path
tokenizer = tokenizer_class.from_pretrained(tokenizer_name_or_path, **tokenizer_kwargs)
# Required for llama
if model_args.model_type == "llama" and tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token": DEFAULT_PAD_TOKEN})

tokenizer

BloomTokenizerFast(name_or_path='../../../models/bigscience/bloomz-560m', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [14]:
if training_args.use_peft:
    if training_args.peft_path is not None:
        logger.info(f"Peft from pre-trained model: {training_args.peft_path}")
        model = PeftModel.from_pretrained(model, training_args.peft_path, is_trainable=True)
    else:
        logger.info("Init new peft model")
        target_modules = training_args.target_modules.split(',') if training_args.target_modules else None
        if target_modules and 'all' in target_modules:
            target_modules = find_all_linear_names(model, int4=False, int8=model_args.load_in_8bit)
        modules_to_save = training_args.modules_to_save
        if modules_to_save is not None:
            modules_to_save = modules_to_save.split(',')
        logger.info(f"Peft target_modules: {target_modules}")
        logger.info(f"Peft lora_rank: {training_args.lora_rank}")
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            target_modules=target_modules,
            inference_mode=False,
            r=training_args.lora_rank,
            lora_alpha=training_args.lora_alpha,
            lora_dropout=training_args.lora_dropout,
            modules_to_save=modules_to_save)
        model = get_peft_model(model, peft_config)
    if model_args.load_in_8bit:
        model = prepare_model_for_int8_training(model)
    model.print_trainable_parameters()
else:
    logger.info("Full parameters training")
    print_trainable_parameters(model)

2023-06-07 17:43:41.896 | INFO     | __main__:<cell line: 1>:6 - Init new peft model
2023-06-07 17:43:41.897 | INFO     | __main__:<cell line: 1>:13 - Peft target_modules: ['dense', 'dense_4h_to_h', 'dense_h_to_4h', 'query_key_value']
2023-06-07 17:43:41.898 | INFO     | __main__:<cell line: 1>:14 - Peft lora_rank: 8


trainable params: 3145728 || all params: 562360320 || trainable%: 0.5593794384354857


## 加载数据集

In [15]:
# Get datasets
if data_args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        data_args.dataset_name,
        data_args.dataset_config_name,
        cache_dir=model_args.cache_dir,
    )
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
        )
        raw_datasets["train"] = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
        )
else:
    data_files = {}
    if data_args.train_file_dir is not None and os.path.exists(data_args.train_file_dir):
        train_data_files = glob(f'{data_args.train_file_dir}/**/*.json', recursive=True) + glob(
            f'{data_args.train_file_dir}/**/*.jsonl', recursive=True)
        logger.info(f"train files: {', '.join(train_data_files)}")
        data_files["train"] = train_data_files
    if data_args.validation_file_dir is not None and os.path.exists(data_args.validation_file_dir):
        eval_data_files = glob(f'{data_args.validation_file_dir}/**/*.json', recursive=True) + glob(
            f'{data_args.validation_file_dir}/**/*.jsonl', recursive=True)
        logger.info(f"eval files: {', '.join(eval_data_files)}")
        data_files["validation"] = eval_data_files
    raw_datasets = load_dataset(
        'json',
        data_files=data_files,
        cache_dir=model_args.cache_dir,
    )
    # If no validation data is there, validation_split_percentage will be used to divide the dataset.
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            'json',
            data_files=data_files,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=model_args.cache_dir,
        )
        raw_datasets["train"] = load_dataset(
            'json',
            data_files=data_files,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=model_args.cache_dir,
        )
logger.info(f"Raw datasets: {raw_datasets}")

2023-06-07 17:43:45.653 | INFO     | __main__:<cell line: 2>:27 - train files: ../data/finetune/Belle_open_source_1k.json
2023-06-07 17:43:45.655 | INFO     | __main__:<cell line: 2>:32 - eval files: ../data/finetune/Belle_open_source_1k.json
Found cached dataset json (/home/flemingxu/.cache/huggingface/datasets/json/default-81868ba195386786/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


  0%|          | 0/2 [00:00<?, ?it/s]

2023-06-07 17:43:46.858 | INFO     | __main__:<cell line: 53>:53 - Raw datasets: DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1000
    })
})


In [16]:
# Preprocessing the datasets
max_source_length = data_args.max_source_length
max_target_length = data_args.max_target_length

def preprocess_function(examples):
    sources = []
    targets = []
    for instruction, input, output in zip(examples['instruction'], examples['input'], examples['output']):
        if input:
            instruction = instruction + '\n' + input
        source = PROMPT_TEMPLATE.format_map({'instruction': instruction})
        target = f"{output}{tokenizer.eos_token}"

        sources.append(source)
        targets.append(target)

    tokenized_sources = tokenizer(sources, truncation=True, max_length=max_source_length)
    tokenized_targets = tokenizer(targets, add_special_tokens=False, truncation=True, max_length=max_target_length)

    all_input_ids = []
    all_labels = []
    for s, t in zip(tokenized_sources['input_ids'], tokenized_targets['input_ids']):
        input_ids = torch.LongTensor(s + t)
        labels = torch.LongTensor([IGNORE_INDEX] * (max_source_length + max_target_length - len(t)) + t)
        all_input_ids.append(input_ids)
        all_labels.append(labels)
    results = {'input_ids': all_input_ids, 'labels': all_labels}

    return results

train_dataset = None
max_train_samples = 0
if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets['train']
    max_train_samples = len(train_dataset)
    if data_args.max_train_samples is not None and data_args.max_train_samples > 0:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))
    logger.debug(f"Example train_dataset[0]: {train_dataset[0]}")
    with training_args.main_process_first(desc="Train dataset tokenization"):
        train_dataset = train_dataset.shuffle().map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=train_dataset.column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )
        logger.debug(f"Num train_samples: {len(train_dataset)}")
        logger.debug("Tokenized training example:")
        logger.debug(tokenizer.decode(train_dataset[0]['input_ids']))

train_dataset

2023-06-07 17:43:46.904 | DEBUG    | __main__:<cell line: 33>:41 - Example train_dataset[0]: {'instruction': '为给定的句子生成一个同义句。\nShe is studying for her final exams.', 'input': '', 'output': 'She is preparing for her last exams.'}


Running tokenizer on dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

2023-06-07 17:43:48.001 | DEBUG    | __main__:<cell line: 33>:51 - Num train_samples: 1000
2023-06-07 17:43:48.002 | DEBUG    | __main__:<cell line: 33>:52 - Tokenized training example:
2023-06-07 17:43:48.003 | DEBUG    | __main__:<cell line: 33>:53 - Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
根据给定的段落，对其进行编辑，使其更清楚或更简洁。
这段话来自一篇科学文章，但表达得不够清楚：
“本次实验的过程包括了获得实验组的样本、随机分配样本、实施处理并记录结果。最后，使用统计学方法分析数据以得出结论。”


### Response: 本实验包括：获取实验样本、随机分配处理、记录实验结果并使用统计学方法分析数据得出结论。</s>


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1000
})

In [17]:
eval_dataset = None
max_eval_samples = 0
if training_args.do_eval:
    with training_args.main_process_first(desc="Eval dataset tokenization"):
        if "validation" not in raw_datasets:
            raise ValueError("--do_eval requires a validation dataset")
        eval_dataset = raw_datasets["validation"]
        max_eval_samples = len(eval_dataset)
        if data_args.max_eval_samples is not None and data_args.max_eval_samples > 0:
            max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
            eval_dataset = eval_dataset.select(range(max_eval_samples))
        logger.debug(f"Example eval_dataset[0]: {eval_dataset[0]}")
        eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=eval_dataset.column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )
        logger.debug(f"Num eval_samples: {len(eval_dataset)}")
        logger.debug("Tokenized eval example:")
        logger.debug(tokenizer.decode(eval_dataset[0]['input_ids']))
        
eval_dataset

2023-06-07 17:43:48.047 | DEBUG    | __main__:<cell line: 3>:12 - Example eval_dataset[0]: {'instruction': '为给定的句子生成一个同义句。\nShe is studying for her final exams.', 'input': '', 'output': 'She is preparing for her last exams.'}


Running tokenizer on dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

2023-06-07 17:43:48.635 | DEBUG    | __main__:<cell line: 3>:21 - Num eval_samples: 10
2023-06-07 17:43:48.636 | DEBUG    | __main__:<cell line: 3>:22 - Tokenized eval example:
2023-06-07 17:43:48.638 | DEBUG    | __main__:<cell line: 3>:23 - Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
为给定的句子生成一个同义句。
She is studying for her final exams.

### Response: She is preparing for her last exams.</s>


Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 10
})

## 指定Trainer

In [18]:
# Initialize our Trainer
if training_args.gradient_checkpointing:
    model.gradient_checkpointing_enable()
    model.config.use_cache = False
else:
    model.config.use_cache = True
model.enable_input_require_grads()

if torch.cuda.device_count() > 1:
    # Keeps Trainer from trying its own DataParallelism when more than 1 gpu is available
    model.is_parallelizable = True
    model.model_parallel = True
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    return_tensors="pt",
    padding="max_length",
    max_length=max_source_length + max_target_length
)
trainer = SavePeftModelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset if training_args.do_train else None,
    eval_dataset=eval_dataset if training_args.do_eval else None,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## 开始训练

In [20]:
# Training
if training_args.do_train:
    logger.info("*** Train ***")
    logger.debug(f"Train dataloader example: {list(trainer.get_train_dataloader())[0]}")
    checkpoint = None

    train_result = trainer.train(resume_from_checkpoint=checkpoint)

    metrics = train_result.metrics

    metrics["train_samples"] = max_train_samples
    logger.debug(f"Training metrics: {metrics}")
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    logger.info(f"Saving model checkpoint to {training_args.output_dir}")
    save_model(training_args.output_dir, model, tokenizer, training_args)

metrics

2023-06-07 17:45:21.604 | INFO     | __main__:<cell line: 2>:3 - *** Train ***
2023-06-07 17:45:22.130 | DEBUG    | __main__:<cell line: 2>:4 - Train dataloader example: {'input_ids': tensor([[     3,      3,      3,  ..., 218591,    420,      2],
        [     3,      3,      3,  ...,  75844,    420,      2],
        [     3,      3,      3,  ...,   8401,    420,      2],
        [     3,      3,      3,  ..., 215090,    420,      2]]), 'labels': tensor([[  -100,   -100,   -100,  ..., 218591,    420,      2],
        [  -100,   -100,   -100,  ...,  75844,    420,      2],
        [  -100,   -100,   -100,  ...,   8401,    420,      2],
        [  -100,   -100,   -100,  ..., 215090,    420,      2]]), 'attention_mask': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]])}
/home/flemingxu/disk/py38/lib/python3.8/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is

Step,Training Loss,Validation Loss
10,2.903400,3.215630
20,2.830500,3.142511
30,2.678200,3.100051
40,2.816300,3.042619
50,2.393300,3.007197
60,2.552900,2.991486
70,2.609200,2.973372
80,2.809700,2.954670
90,2.475300,2.945711
100,2.860300,2.944793


2023-06-07 17:46:26.448 | DEBUG    | __main__:<cell line: 2>:12 - Training metrics: {'train_runtime': 64.2933, 'train_samples_per_second': 7.777, 'train_steps_per_second': 1.944, 'total_flos': 469185331200000.0, 'train_loss': 2.719934181213379, 'epoch': 0.5, 'train_samples': 1000}
2023-06-07 17:46:26.471 | INFO     | __main__:<cell line: 2>:16 - Saving model checkpoint to outputs-sft


***** train metrics *****
  epoch                    =        0.5
  total_flos               =   436962GF
  train_loss               =     2.7199
  train_runtime            = 0:01:04.29
  train_samples            =       1000
  train_samples_per_second =      7.777
  train_steps_per_second   =      1.944


{'train_runtime': 64.2933,
 'train_samples_per_second': 7.777,
 'train_steps_per_second': 1.944,
 'total_flos': 469185331200000.0,
 'train_loss': 2.719934181213379,
 'epoch': 0.5,
 'train_samples': 1000}

In [21]:
# Evaluation
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    metrics = trainer.evaluate()

    metrics["eval_samples"] = max_eval_samples
    try:
        perplexity = math.exp(metrics["eval_loss"])
    except OverflowError:
        perplexity = float("inf")
    metrics["perplexity"] = perplexity
    logger.debug(f"Eval metrics: {metrics}")
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

2023-06-07 17:46:26.778 | INFO     | __main__:<cell line: 2>:3 - *** Evaluate ***


2023-06-07 17:46:27.118 | DEBUG    | __main__:<cell line: 2>:12 - Eval metrics: {'eval_loss': 2.934418201446533, 'eval_runtime': 0.3326, 'eval_samples_per_second': 30.068, 'eval_steps_per_second': 9.021, 'epoch': 0.5, 'eval_samples': 10, 'perplexity': 18.81055599670998}


***** eval metrics *****
  epoch                   =        0.5
  eval_loss               =     2.9344
  eval_runtime            = 0:00:00.33
  eval_samples            =         10
  eval_samples_per_second =     30.068
  eval_steps_per_second   =      9.021
  perplexity              =    18.8106


In [22]:
training_args.output_dir

'outputs-sft'

In [23]:
%ls outputs-sft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
adapter_config.json  runs/                    train_results.json
adapter_model.bin    special_tokens_map.json  trainer_state.json
all_results.json     tokenizer.json           training_args.bin
eval_results.json    tokenizer_config.json


模型训练结果：
- 使用lora训练模型，则保存的lora权重是`adapter_model.bin`, lora配置文件是`adapter_config.json`
- 日志保存在`output_dir/logs`

查看运行日志，默认使用的是tensorboard保存日志，启动方式如下：


```
tensorboard --logdir output_dir/logs --host 0.0.0.0 --port 8009
```

本节完。